## This is just to experiment with the Effect of simulated Vibration

In [1]:
import numpy as np
from glob import glob
from PIL import Image
import cv2

### Calculate the Image Gradient

In [2]:
def imgradient(img):
    if type(img) != np.ndarray:
        img = img.convert('L')
        img = np.asarray(img)

    sobel_x = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
    Gx = cv2.filter2D(img, -1, sobel_x)
    Gy = cv2.filter2D(img, -1, sobel_y)
    grad_mag = np.sqrt(Gx**2 + Gy**2)
    return grad_mag

### Function for Turbulence strength calculation

In [3]:
def fcn_Cnsqr_sequence(data, PFOV, D, L, P):
    sigma_I = np.std(data, axis=0)
    Cn_sqr_vec = []
    for iSequence in range(data.shape[0]):
        d = data[iSequence,:,:]
        Grad_mag = imgradient(d)
        Grad_mag[Grad_mag < Grad_mag.max()/100] = np.nan
        Cn_sqr_mn = np.divide((np.dot(PFOV**2, D**(1/3)) * sigma_I**2), (Grad_mag*float(L)*float(P)))
        Cn_sqr_vec.append(np.nanmean(Cn_sqr_mn))
    return np.nanmean(Cn_sqr_vec)

### Compute the Time interval and unique time

In [4]:
focal_length, fstop = 0.5, 8.000
D, ImageScalledDown, lambda_temp, L = focal_length/fstop, 3, 500E-9, 2789
dx, dy = ImageScalledDown*0.003208/1000, ImageScalledDown*0.003208/1000
PFOV, l_0, L_0 = dx/np.array(focal_length), 1 / 1000, 80
P = 2.914 if D < np.sqrt(lambda_temp * L) and l_0 <= D < L_0 else 1.1

print(f'{PFOV=}, {D=}, {L=}, {P=}')

PFOV=1.9248000000000003e-05, D=0.0625, L=2789, P=1.1


### Estimate the Turbulence intensity

In [5]:
image_set = glob("Input/1e-12/*.png")
gray_images = [Image.open(img_path).convert('L') for img_path in image_set]
data = np.array([np.array(img) for img in gray_images])

cn2 = fcn_Cnsqr_sequence(data, PFOV, D, L, P)
print(f'{cn2=}')

cn2=7.397968920754091e-12
